In [ ]:
pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.3 MB/s 
     |████████████████████████████████| 10.2 MB 41.7 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 140 kB 53.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 25 kB/s 
     |████████████████████████████████| 199 kB 43.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f3fa1741a48fa8ff0ec4a71e8f720bd12a24c699df12cd7d8a958c36f0cc1f45
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf

In [ ]:
import boto3
import os
import pandas as pd
import secrets

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
!ls

data  sample_data  spark-3.1.1-bin-hadoop3.2  spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) 
spark

In [ ]:
data1= pd.read_csv('data/product_202211221632.csv', index_col=0)

In [ ]:
data1

,price,skin_color
product_id,,
1,5,Red
2,10,Blue
3,15,NaN


In [ ]:
data2= pd.read_csv('data/event__202211221632.csv', index_col=0)

In [ ]:
data2

,device_id,event_name,event_time
user_id,,,
User_000058,Device_058,Puzzle_Finish_004,2022-11-01 00:01:00.000 +0300
User_000033,Device_033,Bonus_Activation_001,2022-11-01 00:04:00.000 +0300
User_000078,Device_078,Puzzle_Finish_005,2022-11-01 00:05:00.000 +0300
User_000074,Device_074,Leaderboard_Open,2022-11-01 00:10:00.000 +0300
User_000032,Device_032,Level_Lose,2022-11-01 00:12:00.000 +0300
...,...,...,...
User_000081,Device_081,Ping,2022-11-12 18:24:00.000 +0300
User_000011,Device_011,Puzzle_Finish_001,2022-11-16 13:24:00.000 +0300
User_000063,Device_063,Puzzle_Finish_005,2022-11-01 22:43:00.000 +0300


In [ ]:
data3= pd.read_csv('data/age_segment_202211221632.csv', index_col=0)

In [ ]:
data3

,age_min,age_max
id,,
1,0,18
2,19,24
3,25,30
4,31,40
5,41,55


In [ ]:
data4= pd.read_csv('data/account_202211221632.csv', index_col=0)

In [ ]:
data4

,device_id,age,gender
user_id,,,
User_000001,Device_001,44,Female
User_000002,Device_002,46,Female
User_000003,Device_003,24,Male
User_000004,Device_004,47,Female
User_000005,Device_005,29,Male
...,...,...,...
User_000076,Device_076,23,Female
User_000077,Device_077,39,Male
User_000078,Device_078,41,Female


In [ ]:
from numpy import savetxt

In [ ]:
data.duplicated()

user_id
User_000058    False
User_000033    False
User_000078    False
User_000074    False
User_000032    False
               ...  
User_000081    False
User_000011    False
User_000063    False
User_000054    False
User_000076    False
Length: 22720, dtype: bool

In [ ]:
account_df=spark.read.csv("data/account_202211221632.csv", header=True, inferSchema=True)
account_df.coalesce(1).write.mode ("overwrite").format("com.databricks.spark.csv").option("header", "true").save("filename")

age_df=spark.read.csv("data/age_segment_202211221632.csv", header=True, inferSchema=True)
age_df.coalesce(1).write.mode ("overwrite").format("com.databricks.spark.csv").option("header", "true").save("filename")

event_df=spark.read.csv("data/event__202211221632.csv", header=True, inferSchema=True)
event_df.coalesce(1).write.mode ("overwrite").format("com.databricks.spark.csv").option("header", "true").save("filename")

product_df=spark.read.csv("data/product_202211221632.csv", header=True, inferSchema=True)
product_df.coalesce(1).write.mode ("overwrite").format("com.databricks.spark.csv").option("header", "true").save("filename")


In [ ]:
account_df.printSchema()
age_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- age_min: integer (nullable = true)
 |-- age_max: integer (nullable = true)



In [ ]:
account_df.createOrReplaceTempView("account")
age_df.createOrReplaceTempView("age")
event_df.createOrReplaceTempView("event")
product_df.createOrReplaceTempView("product")


In [ ]:
query_data =spark.sql('SELECT ac.user_id, ac.device_id, evt.event_time, ac.age, ac.gender FROM event evt INNER JOIN account ac on evt.user_id=ac.user_id  ')
query_data




user_id,device_id,event_time,age,gender
User_000058,Device_058,2022-10-31 21:01:00,27,Male
User_000033,Device_033,2022-10-31 21:04:00,28,Male
User_000078,Device_078,2022-10-31 21:05:00,41,Female
User_000074,Device_074,2022-10-31 21:10:00,22,Female
User_000032,Device_032,2022-10-31 21:12:00,21,Male
User_000015,Device_015,2022-11-02 19:54:00,32,Female
User_000035,Device_035,2022-11-06 14:49:00,45,Male
User_000039,Device_039,2022-11-09 08:35:00,32,Female
User_000070,Device_070,2022-11-11 14:55:00,34,Male
User_000031,Device_031,2022-11-11 15:51:00,24,Male


In [ ]:
query_data.write.csv("rettention_graf2")

In [ ]:
query_data =spark.sql('SELECT * FROM event WHERE device_id="Device_058" ' )
query_data

user_id,device_id,event_name,event_time
User_000058,Device_058,Puzzle_Finish_004,2022-10-31 21:01:00
User_000058,Device_058,App_Purchase_003,2022-11-02 09:52:00
User_000058,Device_058,Bonus_Activation_003,2022-11-11 15:30:00
User_000058,Device_058,Puzzle_Finish_002,2022-11-03 14:00:00
User_000058,Device_058,Puzzle_Finish_003,2022-11-09 02:30:00
User_000058,Device_058,Puzzle_Finish_002,2022-11-10 02:04:00
User_000058,Device_058,Puzzle_Finish_001,2022-11-10 02:50:00
User_000058,Device_058,Bonus_Activation_003,2022-11-08 16:33:00
User_000058,Device_058,Puzzle_Finish_002,2022-11-10 00:29:00
User_000058,Device_058,Puzzle_Finish_001,2022-11-01 20:12:00


In [ ]:
queryy_data =spark.sql('SELECT age, Count(device_id) FROM account GROUP BY account.age ORDER BY age' )
queryy_data


age,count(device_id)
18,2
20,1
21,4
22,5
23,2
24,2
25,1
27,3
28,4
29,2
